Introduction



Import statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime as dt
import math

here = os.getcwd()

Load the 2020 data set and the reference files.

In [2]:
a = pd.read_csv('data/2020/2020datasimp.csv')
b = pd.read_csv('data/2020/map-keys-2020.csv')
c = pd.read_csv("refs/florahelvetica/florahelveticarcsvnew.csv")


In [3]:
b.rename(columns={'X':'lon', 'Y':'lat', 'Name':'place'}, inplace=True)
a.rename(columns={'sci':'species'}, inplace=True)

In [3]:
replacedict = {
    'verbanum bonariensis ':'verbena bonariensis',
    'medicago varia':'medicago sativa',
    "oenothera":"oenothera biennis",
    "geranium pratens":"geranium pratense",
    "senecio jacobaea": "jacobaea vulgaris",
    "oenothera biennis ": "oenothera biennis",
    "oenothera biennis agg.": "oenothera biennis",
    "solidalgo canadensis": "solidago canadensis",
    "verbascum lynchitis":"verbascum lychnitis",
    "verbascum negris":"verbascum nigrum",
    "securigea varia": "securigera varia",
    "melilotus officianalis": "melilotus officinalis",
    "knautia maxima": "knautia dipsacifolia",
    "hieracium aurantiacum":"pilosella aurantiaca",
    "sysimbrium officinale":"sisymbrium officinale",
    "geranium robertanium":"geranium robertianum",
    "mycelis muralis": "lactuca muralis",
    "calamintha nepeta":"clinopodium nepeta",
    "polygonum persicaria":"persicaria maculosa",
    "sorbus aria":"aria edulis",
    "taraxacum": "taraxacum officinale"
}


In [5]:
def new_func(x,keys):
    try:
        data = keys[x]
    except:
        data = x
    return data
a["species1"] = a.species.map(lambda x: new_func(x, replacedict))

# b["species"] = b.species1

In [6]:
a['string_list'] = a.species1.str.split( " ")
a['string_list'] = a.string_list.map(lambda x: [name for name in x if name != 'agg.'])

In [7]:
a['string_list']

0      [plantago, lanceolata]
1          [centaurea, nigra]
2           [plantago, media]
3        [chenopodium, album]
4          [centaurea, jacea]
                ...          
930       [allium, carinatum]
931       [viburnum, lantana]
932    [filipendula, ulmaria]
933      [lythrum, salicaria]
934    [lysimachia, vulgaris]
Name: string_list, Length: 935, dtype: object

In [8]:
a['slug'] = a.string_list.map(lambda x: "-".join(x))
a['species'] = a.string_list.map(lambda x: " ". join(x))
a['place'] = a.place.map(lambda x:  x.replace(" ","-"))

In [9]:
def change_string(x):
    try:
        s_data = x.split('.')
        data = s_data[::-1]
        data = "-".join(data)
    except:
        print("no luck")
        data = x
    
    return data
a['new_date'] = a.date.map(lambda x: change_string(x))

In [10]:
def make_timestamp(x):
    try:        
        data = dt.datetime.strptime(x, "%Y-%m-%d")        
    except:        
        data = 'no date'
    
    return data
a['check_date'] = a.new_date.map(lambda x: make_timestamp(x))
a['date'] = a.new_date

print(F"\nThese are the records in a with no valid date:\n\n{a[a.check_date == 'no date']}\n")


These are the records in a with no valid date:

Empty DataFrame
Columns: [place, date, time, species, name, species1, string_list, slug, new_date, check_date]
Index: []



In [11]:
a_to_csv = a[['place','date','species', 'name', 'slug']].copy()
a_not_in_c = [x for x in a_to_csv.species.unique() if x not in c.species.unique()]
print(F"\nThese are the species in a that do not match c:\n\n{a_not_in_c }\n")
len(a_not_in_c)


These are the species in a that do not match c:

['erigeron annuus', 'taraxacum officinale', 'rubus rubus sect.', 'tripleurospermum maritimum', 'persicaria longiseta', 'ligustrum vulgare', 'anemone hupehensis', 'packera aurea', 'dipsacus fullonum', 'rubus fruticosus', 'ligustrum sinense', 'eupatorium serotinum', 'geranium pyrenaicum ', 'salix nigra', 'phlox pilosa', 'hieracium morulum']



16

In [12]:
a_to_csv

,place,date,species,name,slug
0,alleestrasse-1,2020-09-02,plantago lanceolata,plantain lancéolé,plantago-lanceolata
1,alleestrasse-1,2020-09-02,centaurea nigra,centaurée noire,centaurea-nigra
2,alleestrasse-1,2020-09-02,plantago media,plantain moyen,plantago-media
3,alleestrasse-1,2020-09-02,chenopodium album,chénopode blanc,chenopodium-album
4,alleestrasse-1,2020-09-02,centaurea jacea,centaurée jacée,centaurea-jacea
...,...,...,...,...,...
930,weissenau,2020-08-10,allium carinatum,ail caréné,allium-carinatum
931,weissenau,2020-08-10,viburnum lantana,"mancienne, viorne lantane",viburnum-lantana
932,weissenau,2020-08-10,filipendula ulmaria,"reine des prés, spirée",filipendula-ulmaria
933,weissenau,2020-08-10,lythrum salicaria,salicaire commune,lythrum-salicaria


In [13]:
b['place'] = b.place.str.split('\n').str[0]
b.place = b.place.str.lower()
b.place = b.place.map(lambda x:  x.replace(" ","-"))


In [14]:
# saving the location map keys
afilename= F"{here}/data/2020/map-keys-2020.csv"
b.to_csv(afilename, index=False)

In [15]:
afilename = here+"/data/2020/species2020_matching.csv"
afilename2 = here+"/data/2020/species2020_full.csv"

matching_species = a_to_csv[~a_to_csv.species.isin(a_not_in_c)]

a_to_csv.to_csv(afilename2, index=False)

matching_species.to_csv(afilename, index=False)

In [16]:
# find place names that don't match:
a_locs = a.place.unique()
b_locs = b.place.unique()
a_species = a.species.unique()
c_species = c.species.unique()

# get the symetric difference between the different place name lists
placenames = list(set(a_locs) | set(b_locs))
intersect_placenames = list(set(a_locs) & set(b_locs))
print(F"There are {len(placenames)} different place names\n")
print(F"\nThere are {len(a_locs)} different names in a\n")
print(F"\nThere are {len(b_locs)} different names in b\n")
print(F"\nThere are {len(intersect_placenames)} shared between a and b")

#find the species in the data set that are not in the flora helvetica set
speciesnames = list(set(a_species) | set(c_species))
intersect_speciesnames = list(set(a_species) & set(c_species))
in_b_not_d = list(set(a_species) - set(intersect_speciesnames))
print(F"There are {len(speciesnames)} different species names\n")
print(F"\nThere are {len(a_species)} different names in a\n")
print(F"\nThere are {len(c_species)} different names in c\n")
print(F"\nThere are {len(intersect_speciesnames)} shared between a and c")
print(F"\nThere are {len(a_not_in_c)} in a not in c")

There are 111 different place names


There are 111 different names in a


There are 111 different names in b


There are 111 shared between a and b
There are 3434 different species names


There are 232 different names in a


There are 3418 different names in c


There are 216 shared between a and c

There are 16 in a not in c
